# Text Classification Exploration Notebook

Goal: explore and build text classification models for the news dataset, from simple TF‑IDF baselines to tuned pipelines and final submission files.

Main steps:
- Load and inspect the labeled training data.
- Build a combined text field from title and article.
- Create a train/validation split for fair evaluation.
- Convert text to TF‑IDF features and train baseline models.
- Tune key hyperparameters (C, n‑grams, min_df, etc.).
- Define a final pipeline, train it on all data, and generate competition submissions.

In each numbered section below, the markdown explains:
- what the following code is trying to achieve (aim), and
- how to interpret the printed results or outputs.

In [5]:
import numpy as np
import pandas as pd
import warnings

# Suppress FutureWarning messages from libraries (e.g., scikit-learn)
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

### 1) Load and inspect data

**Aim:** Read the labeled news articles from `development.csv` and quickly understand the basic structure of the dataset and the distribution of labels.

- The next code cells load the CSV into a DataFrame `df` and display `.head()`, `.columns`, and `.info()`.
- The `df['label'].value_counts()` cell shows how many examples there are for each class and highlights any imbalance.

**What the results show:**
- That the data file can be loaded without errors and how many rows/columns it contains.
- Which columns are available (e.g. title, article, label) and their dtypes.
- That some classes are more or less frequent, motivating later steps on class imbalance and robust evaluation.

In [6]:


df = pd.read_csv("development.csv")
df.head()
df.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79997 entries, 0 to 79996
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         79997 non-null  int64 
 1   source     79997 non-null  object
 2   title      79996 non-null  object
 3   article    79996 non-null  object
 4   page_rank  79997 non-null  int64 
 5   timestamp  79997 non-null  object
 6   label      79997 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 4.3+ MB


In [7]:
df['label'].value_counts()

label
0    23542
5    13053
2    11161
1    10588
3     9977
4     8574
6     3102
Name: count, dtype: int64

### 2) Build text feature (title + article)

**Aim:** Create a single text field that concatenates `title` and `article` so the model can learn from both in one go, while safely handling missing values.

- The following code fills missing titles/articles with empty strings and joins them with a space into a new `text` column.
- Viewing `df['text'].head()` is a quick sanity check that the combined text looks reasonable.

**What the results show:**
- Each row now has a clean, combined text representation ready for vectorization.
- There are no `NaN` values or obvious concatenation issues (such as double spaces or missing parts).

replaces missing titles with empty strings
prevents error during string concatention
adds space

In [8]:
df['text'] = df['title'].fillna('') + ' ' + df['article'].fillna('')
df['text'].head()

0    OPEC Boosts Nigeria&#39;s Oil Revenue By .82m ...
1    Yearender: Mideast peace roadmap reaches dead-...
2    Battleground Dispatches for Oct. 5 \\n    (CQP...
3    Air best to resuscitate newborns Air rather th...
4    High tech German train crash kills at least on...
Name: text, dtype: object

### 3) Train / validation split

**Aim:** Split the text data into training and validation sets so that models are evaluated on unseen examples in a fair, reproducible way.

- `X_text` selects the input texts and `y` selects the labels.
- `train_test_split(..., test_size=0.2, random_state=42, stratify=y)` creates an 80/20 split while preserving class proportions in both splits.
- The variables `X_train_text`, `X_val_text`, `y_train`, and `y_val` are then used for model training and validation.

**What the results show:**
- The split succeeds and produces separate training and validation subsets.
- Thanks to `stratify=y`, even minority classes appear in the validation set, making the later Macro F1 scores meaningful.

X_texts selects the input, in supervised learning
y -> selects the target value, in supervised learning, what the model must predict

In [9]:
X_text = df["text"]
y = df["label"]

X_train_text
text used to train the model

y_train
labels used to learn

X_val_text
text the model never sees during training

y_val
true labels used only for evaluation


In [10]:
from sklearn.model_selection import train_test_split

X_train_text, X_val_text, y_train, y_val = train_test_split(
    X_text,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

EXPLORATORY BASELINE DISCARDED 3) Baseline approach using TF-IDF + Logistic Regression

text classification
removes very common words with stop-words
we dont want to overfit so we should choose a proper max_features

fit_transformer: fit-> learns the vocab, how frewuent they are and their IDF weight,

transform -> converts each document into a TF-IDF vector
each row: article
each column: one word
each value: importance of that word in the article

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    stop_words="english",
    max_features=50000
)

X_tfidf = vectorizer.fit_transform(df["text"])
X_tfidf.shape

(79997, 50000)

Raw text (title + article)
↓
TF-IDF vectors (numerical, sparse, high-dimensional)
↓
Ready for LinearSVC / LogisticRegression

NOT: tf-ıdf is done on all the data (train + val)

we will fix it later

Description: 
Rows = articles
Columns = words (features)

🔹 79997 → number of articles
You have 79,997 news articles in development.csv.
Each row corresponds to one article.

🔹 50000 → number of features (words)
You told TF-IDF:
	•	The system selected the 50,000 most important words/word-patterns
	•	Each column corresponds to one word (or word combination)

One Row looks like this: [0.0, 0.12, 0.0, 0.87, 0.03, 0.0, ...]

This line confirms that:
	•	✅ Your text → numbers conversion worked
	•	✅ You now have a valid ML input
	•	✅ Each article is represented consistently

This is a big milestone, even if it looks simple.

TRAIN VALIDATION SPLIT

In [12]:
from sklearn.model_selection import train_test_split

X = X_tfidf          # features (numbers)
y = df['label']     # target labels

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_val.shape

((63997, 50000), (16000, 50000))

X = X_tfidf
y = df['label']
What this means:
	•	X → the input features
	•	Here: the TF-IDF matrix (numbers representing article text)
	•	y → the target variable
	•	The label (0–6) indicating the news category

In ML notation:
	•	X = inputs
	•	y = correct answers

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)
1️⃣ Splits the data
	•	X_train, y_train → used to train the model
	•	X_val, y_val → used to evaluate the model on unseen data
2️⃣ test_size=0.2
	•	20% of the data goes to validation
	•	80% remains for training
This is a standard choice in the lectures.
3️⃣ random_state=42
	•	Fixes the randomness of the split
	•	Ensures reproducibility
	•	Running the code again gives the same split

Very important for:
	•	debugging
	•	fair comparison of models
4️⃣ stratify=y
	•	Keeps the class proportions the same in train and validation
	•	Important because the dataset is imbalanced (e.g. Health is rare)

Without this:
	•	validation set could miss rare classes
	•	evaluation would be misleading

X_train.shape, X_val.shape
What this checks:
	•	Confirms the split worked
	•	Shows how many samples are in each set
	•	Number of columns (features) stays the same



In [13]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    max_iter=1000,
    n_jobs=-1
)

model.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul


Each row is a hyperparameter of Logistic Regression.

Hyperparameters are:
	•	chosen by you
	•	fixed before training
	•	they control how the model learns
    


🔹 solver = 'lbfgs'

This is the optimization algorithm used to find the weights.

Plain English:

This is the math engine that adjusts the model until it fits the data.

Why this is good:
	•	lbfgs is standard
	•	works well for multiclass classification
	•	handles many features (like TF-IDF)

✔️ Fully aligned with course defaults.

🔹 max_iter = 1000

This is very important.

Plain English:

Maximum number of steps the optimizer is allowed to take.

Why we increased it:
	•	TF-IDF has 50,000 features
	•	Default (100) is often not enough
	•	1000 prevents premature stopping

✔️ Correct and recommended.

🔹 C = 1.0

This controls regularization strength.

Plain English:
	•	Large C → model fits data more closely
	•	Small C → model is more conservative

C = 1.0 means:

“Use a balanced, default amount of regularization.”

✔️ Perfect baseline choice
We’ll maybe tune this later, not now.

🔹 penalty = 'deprecated'

This looks scary but it is not a problem.

What it really means:
	•	You did not explicitly set a penalty
	•	The solver default (l2) is used

So effectively:

You are using L2 regularization, which is standard.

You can safely ignore this for now.

🔹 class_weight = None

This means:

All classes are treated equally during training.

Is this okay?
	•	Yes, for a baseline
	•	Later we may try class_weight='balanced' as an improvement

Right now:
✔️ Totally fine.


🔹 n_jobs = -1

Plain English:

Use all available CPU cores.

This only affects speed, not results.

✔️ Good practice.


	•	LogisticRegression(...)
→ creates the model object
	•	max_iter=1000
→ allows more training iterations so the model converges
(important with many features like TF-IDF)
	•	n_jobs=-1
→ uses all available CPU cores (faster)
	•	model.fit(X_train, y_train)
→ this is where learning happens
The model finds patterns linking word features to labels

4) Baseline evaluation (Macro F1)

In [13]:
y_val_pred = model.predict(X_val)

In [14]:
from sklearn.metrics import f1_score

f1_macro = f1_score(y_val, y_val_pred, average="macro")
f1_macro

0.6475014177285618


0.6475 means:
On unseen validation data, your model is doing a reasonably good job at correctly classifying articles across all 7 categories, giving equal importance to each category.


“The baseline Logistic Regression model achieves a Macro F1 of approximately 0.65 on the validation set, indicating that it generalizes reasonably well across all news categories, including underrepresented ones. This confirms that the feature extraction and learning pipeline is correct.”

5) Handle class imbalance (class_weight)

In [15]:
from sklearn.linear_model import LogisticRegression

model_balanced = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

model_balanced.fit(X_train, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",'balanced'
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :ter

In [16]:
from sklearn.metrics import f1_score

y_val_pred_balanced = model_balanced.predict(X_val)
f1_macro_balanced = f1_score(y_val, y_val_pred_balanced, average="macro")
f1_macro_balanced

0.6600408599457183

In [17]:
from sklearn.metrics import f1_score

def quick_eval(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    return f1_score(y_val, pred, average="macro")

6) Hyperparameter tuning
- C tuning
- TF-IDF ngram_range tuning
- TF-IDF min_df tuning
- (placeholder for max_df, max_features later)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

Cs = [0.25, 0.5, 1, 2, 4, 8]

for C in Cs:
    pipe = Pipeline([
        ("tfidf", TfidfVectorizer(
            stop_words="english",
            ngram_range=(1,2),
            min_df=2,
            max_df=1.0,          # don’t lock 0.6 unless proven
            sublinear_tf=True,   # add this
            max_features=100000
        )),
        ("clf", LogisticRegression(
            C=C, max_iter=2000, class_weight="balanced", n_jobs=-1
        ))
    ])
    scores = cross_val_score(pipe, X_train_text, y_train, scoring="f1_macro", cv=cv)
    print("C=", C, "mean=", scores.mean(), "std=", scores.std())

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


C= 0.25 mean= 0.6521822851941533 std= 0.0018349588448387715


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


C= 0.5 mean= 0.6618358807948415 std= 0.0013555367375706737


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


...

C= 2 mean= 0.6678991571840923 std= 0.0034437685098182578


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


C= 4 mean= 0.6656707341117357 std= 0.00210675895660936


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


C= 8 mean= 0.6611166785622088 std= 0.0028619064054469148


### 6.1) C tuning for LogisticRegression

We evaluate several values of `C` (regularization strength) using Stratified K‑Fold with Macro F1 to find a good trade‑off between under‑ and over‑fitting.

C decided 2

In [24]:
pipe.set_params(clf__C=2)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [25]:
configs = [
    {"tfidf__ngram_range": (1,1)},
    {"tfidf__ngram_range": (1,2)},
    {"tfidf__ngram_range": (1,3)},
]

for cfg in configs:
    pipe.set_params(**cfg)
    pipe.fit(X_train_text, y_train)
    preds = pipe.predict(X_val_text)
    f1 = f1_score(y_val, preds, average="macro")
    print(cfg, "→ Macro F1:", round(f1, 5))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


{'tfidf__ngram_range': (1, 1)} → Macro F1: 0.65544


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


{'tfidf__ngram_range': (1, 2)} → Macro F1: 0.67574


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


{'tfidf__ngram_range': (1, 3)} → Macro F1: 0.67781


### 6.2) Tuning TF‑IDF ngram_range

We compare different `ngram_range` settings for `TfidfVectorizer` (e.g. `(1,1)`, `(1,2)`, `(1,3)`) and measure Macro F1 to see which mixture of unigrams/bigrams/trigrams works best.

ngram_range : (1,2)

In [30]:
configs = [
    {"tfidf__min_df": 3},
    {"tfidf__min_df": 4},
    {"tfidf__min_df": 5},
]

for cfg in configs:
    pipe.set_params(tfidf__ngram_range=(1,3), **cfg)
    pipe.fit(X_train_text, y_train)
    preds = pipe.predict(X_val_text)
    f1 = f1_score(y_val, preds, average="macro")
    print(cfg, "→ Macro F1:", round(f1, 5))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


{'tfidf__min_df': 3} → Macro F1: 0.67781


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


{'tfidf__min_df': 4} → Macro F1: 0.67473


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


{'tfidf__min_df': 5} → Macro F1: 0.67409


### 6.3) Tuning TF‑IDF min_df

We vary `min_df` (the minimum document frequency threshold) to control how rare words are filtered out and observe how this affects Macro F1.

min_df decided 2


In [27]:
pipe.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(max_features=100000, min_df=3, ngram_range=(1, 3),
                   stop_words='english', sublinear_tf=True)),
  ('clf',
   LogisticRegression(C=2, class_weight='balanced', max_iter=2000, n_jobs=-1))],
 'transform_input': None,
 'verbose': False,
 'tfidf': TfidfVectorizer(max_features=100000, min_df=3, ngram_range=(1, 3),
                 stop_words='english', sublinear_tf=True),
 'clf': LogisticRegression(C=2, class_weight='balanced', max_iter=2000, n_jobs=-1),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': 100000,
 'tfidf__min_df': 3,
 'tfidf__ngram_range': (1, 3),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': 'english',
 'tfidf__strip_accents': None,
 'tfidf__sublinear_t

Tuning max_df

In [ ]:
from sklearn.metrics import f1_score

max_dfs = [0.6, 0.7, 0.75, 0.8, 0.9, 0.95, 1.0]

for md in max_dfs:
    pipe.set_params(
        tfidf__ngram_range=(1,3),   # (1,3) seçtiysen bunu sabitle
        tfidf__min_df=2,
        tfidf__max_df=md,
        tfidf__sublinear_tf=True,
        clf__C=2,
        clf__class_weight="balanced"
    )
    pipe.fit(X_train_text, y_train)
    preds = pipe.predict(X_val_text)
    f1 = f1_score(y_val, preds, average="macro")
    print(f"max_df={md:<4}  macroF1={f1:.5f}  | C={pipe.get_params()['clf__C']} ngram={pipe.get_params()['tfidf__ngram_range']}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


max_df=0.6   macroF1=0.67819  | C=2 ngram=(1, 3)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


max_df=0.7   macroF1=0.67819  | C=2 ngram=(1, 3)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


...

max_df=0.9   macroF1=0.67819  | C=2 ngram=(1, 3)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


max_df=0.95  macroF1=0.67819  | C=2 ngram=(1, 3)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


max_df=1.0   macroF1=0.67819  | C=2 ngram=(1, 3)


max_df increased the f1 score but all the versions are the same

You are tuning max_df while keeping fixed:
	•	ngram_range=(1,2)
	•	min_df=2
	•	sublinear_tf=True

In [24]:
from sklearn.metrics import f1_score

max_feats = [20000, 50000, 100000, None]
results = []

for mf in max_feats:
    pipe.set_params(
        tfidf__ngram_range=(1,2),
        tfidf__min_df=2,
        tfidf__max_df=0.6,
        tfidf__sublinear_tf=True,
        tfidf__max_features=mf,
    )
    pipe.fit(X_train_text, y_train)
    preds = pipe.predict(X_val_text)
    f1 = f1_score(y_val, preds, average="macro")
    results.append((mf, f1))
    print(f"max_features={str(mf):<6}  MacroF1={f1:.5f}")

best = max(results, key=lambda x: x[1])
print("\nBEST max_features:", best)

max_features=20000   MacroF1=0.67174
max_features=50000   MacroF1=0.67567
max_features=100000  MacroF1=0.67602
max_features=None    MacroF1=0.67491

BEST max_features: (100000, 0.6760192182529935)


In [31]:
from sklearn.metrics import f1_score

Cs = [1, 2, 4]
for C in Cs:
    pipe.set_params(
        tfidf__ngram_range=(1,3),
        tfidf__min_df=3,
        tfidf__sublinear_tf=True,
        tfidf__max_features=100000,   # hız istersen 50000 yap
        clf__C=C,
        clf__class_weight="balanced"
    )
    pipe.fit(X_train_text, y_train)
    preds = pipe.predict(X_val_text)
    f1 = f1_score(y_val, preds, average="macro")
    print("C=", C, "macroF1=", round(f1, 5))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


C= 1 macroF1= 0.67748


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


C= 2 macroF1= 0.67781


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


C= 4 macroF1= 0.66936


### LinearSVC (SVC) hyperparameter tuning

We now tune the `C` parameter of a `LinearSVC` model with the same TF‑IDF settings as before and compare its macro F1 scores to the LogisticRegression results.

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score

Cs = [0.5, 1.0, 2.0]

for C in Cs:
    pipe = Pipeline([
        ("tfidf", TfidfVectorizer(
            stop_words="english",
            ngram_range=(1,3),
            min_df=3,
            sublinear_tf=True,
            max_features=100000
        )),
        ("clf", LinearSVC(C=C, class_weight="balanced"))
    ])
    pipe.fit(X_train_text, y_train)
    pred = pipe.predict(X_val_text)
    f1 = f1_score(y_val, pred, average="macro")
    print(f"LinearSVC C={C} -> macroF1={f1:.5f}")

LinearSVC C=0.5 -> macroF1=0.67213
LinearSVC C=1.0 -> macroF1=0.66228
LinearSVC C=2.0 -> macroF1=0.65115


**Observation:** The best macro F1 we obtain with this LinearSVC tuning is still lower than the best LogisticRegression score above, so we keep LogisticRegression as our main model.

### Character-level TF-IDF + Logistic Regression

We now try a character-level TF-IDF representation (3–5 character n-grams) with a balanced LogisticRegression classifier and evaluate its macro F1 score on the validation set.

In [35]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

pipe_char = Pipeline([
    ("tfidf", TfidfVectorizer(
        analyzer="char_wb",
        ngram_range=(3,5),
        min_df=3,
        sublinear_tf=True,
        max_features=200000
    )),
    ("clf", LogisticRegression(
        C=2,
        max_iter=2000,
        class_weight="balanced"
    ))
])

pipe_char.fit(X_train_text, y_train)
pred = pipe_char.predict(X_val_text)
f1 = f1_score(y_val, pred, average="macro")
f1

0.6820307304574895

**Comment:** This character-level TF-IDF + LogisticRegression model is mainly used as an alternative view of the text. We compare its macro F1 score with the word-level model and later use it as a building block for possible word+char ensembles, but our primary reference model remains the tuned word-level LogisticRegression.

7) Final pipeline for submission

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

final_pipe = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words="english",
        ngram_range=(1,3),
        min_df=3,
        sublinear_tf=True,
        max_features=100000
    )),
    ("clf", LogisticRegression(
        C=2,
        max_iter=2000,
        class_weight="balanced"
    ))
])

### 7) Define final pipelines for submission

**Aim:** Set up the final text classification pipelines that will be used for training on all data and for generating submissions.

- `final_pipe` is a single TF‑IDF + LogisticRegression pipeline on word n‑grams, used later for the main submission.
- `word_pipeline` and `char_pipeline` mirror the tuned settings for word‑level and character‑level models, useful for ensemble experiments.

**What the results show:**
- These cells only define pipeline objects; there is no output yet. The real effect is that you have reusable, well‑configured models ready to be fitted.

In [36]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

word_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words="english",
        ngram_range=(1, 3),
        min_df=3,
        sublinear_tf=True,
        max_features=100000
    )),
    ("clf", LogisticRegression(
        C=2.0,
        max_iter=2000,
        class_weight="balanced"
    ))
])

In [37]:
char_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        analyzer="char_wb",
        ngram_range=(3, 5),
        min_df=3,
        sublinear_tf=True,
        max_features=200000
    )),
    ("clf", LogisticRegression(
        C=2.0,
        max_iter=2000,
        class_weight="balanced"
    ))
])

### 8) Train final model on full data

**Aim:** Fit `final_pipe` on the **entire** labeled dataset so the model benefits from all available training examples before predicting on the evaluation set.

- `X_text_all` and `y_all` reuse the combined text and labels for every row in `df`.
- `final_pipe.fit(X_text_all, y_all)` trains the TF‑IDF + LogisticRegression model end‑to‑end.

**What the results show:**
- Successful completion of the `fit` call indicates that the final model is trained on full data and ready to be applied to `evaluation.csv`.

In [39]:
X_text_all = df["text"]          # already title+article
y_all = df["label"]

final_pipe.fit(X_text_all, y_all)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


### 9) Generate submission.csv (Id, Predicted)

**Aim:** Apply the trained final model to the unlabeled evaluation set and export predictions in the competition’s required CSV format.

- The next code loads `evaluation.csv`, builds the `text` field in the same way as for training, and uses `final_pipe.predict` to obtain labels.
- It then constructs a DataFrame with columns `Id` and `Predicted` and writes one or more CSV files (e.g. `submission_v1.csv`, `submission.csv`).

**What the results show:**
- Calls like `submission.head()` let you visually confirm that Ids and predicted labels look sensible.
- The final print statements show the number of rows and columns in the saved submission(s), acting as a shape and format sanity check before upload.

In [ ]:
import pandas as pd

eval_df = pd.read_csv("evaluation.csv")
eval_df["text"] = eval_df["title"].fillna("") + " " + eval_df["article"].fillna("")

**Details for evaluation text and submission creation**

- `eval_df = pd.read_csv("evaluation.csv")` loads the competition test set.
- `eval_df["text"] = ...` constructs the same `text` representation (title + article) that was used to train `final_pipe`.
- `submission.to_csv(...)` writes predictions to disk; having both `submission_v1.csv` and `submission.csv` gives you an early version and a final version with the exact same content.

These cells do not change model accuracy; they ensure consistent preprocessing and a correctly formatted output file.

In [ ]:
eval_pred = final_pipe.predict(eval_df["text"])

submission = pd.DataFrame({
    "Id": eval_df["Id"],
    "Predicted": eval_pred
})

submission.to_csv("submission_v1.csv", index=False)
submission.head()

In [ ]:
print("FINAL PIPELINE USED:")
print(final_pipe)

submission = pd.DataFrame({
    "Id": eval_df["Id"],
    "Predicted": eval_pred
})

submission.to_csv("submission.csv", index=False)
print("Saved submission.csv with columns:", submission.columns.tolist())
print("Rows:", len(submission))

In [ ]:
print("evaluation rows:", len(eval_df))
print("submission rows:", len(submission))
print("columns:", submission.columns.tolist())
print(submission["Predicted"].unique())